# IV Workshop on Data and Knowledge Engineering - INFONOR 2020
## **Título:** Redes neuronales recurrentes de corto y largo plazo


**Expositor:** Dr. Juan Bekios Calfa
**Ayudante:** Sr. Ricardo Pizarro


<pre>Universidad Católica del Norte
Departamento de Ingeniería de Sistemas y Computación
Núcleo de Investigación en Inteligencia Artificial y Data Science
</pre>

Referencia:

NLP From Scratch: Classifying Names with a Character-Level RNN

**Autor:** Sean Robertson <https://github.com/spro/practical-pytorch>_

##1. Obtener base de datos

Obtenemos la base de datos sugerida en la referencia.




In [ ]:
%matplotlib inline

**Cargar los datos de Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. Preparando los 
Bajar la base de datos de apellidos `aquí <https://download.pytorch.org/tutorial/data.zip>`.

Extraer estos en el directorio actual.

En ``data/names`` se encuentran 18 textos en archivos llamados "[lenguaje].txt". Cada archivo contiene apellidos, uno por línea, la mayoría normalizados (pero deben ser convertidos de Unicode a ASCII).

Terminaremos con un diccionario con listas de nombres por idioma,
``Idioma: [nombres ...]}``. Las variables genéricas "categoría" y "línea"
(para el lenguaje y el nombre en nuestro caso) se utilizan para su posterior ampliación.

In [ ]:
import unicodedata
import string
import glob # Esta instruccion para los nombres de un archivo dado un path
import os

PATH_TEXT = '/content/drive/My Drive/D-UCN/Classes/TecnicasAvanzadasAprendizajeAutomatico/Laboratorios/Laboratorio06:RedesRecurrentes/database/names'

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

print(n_letters)

# Convertir el código Unicode al texto ASCII (https://stackoverflow.com/a/518232/2809427)
def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn'
      and c in all_letters
  )

#print(unicodeToAscii('Ślusàrski'))

# Leer los archivos y los cargaremos en un estructura de tipo diccionario
category_lines = {}
all_categories = []

def readLines(filenames):
  lines = open(filename, encoding='utf-8').read().strip().split('\n')
  return [unicodeToAscii(line) for line in lines]

for filename in glob.glob(f'{PATH_TEXT}/*.txt'):
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = readLines(filename)
  category_lines[category] = lines

n_categories = len(all_categories)
print(n_categories)

Ahora tenemos `category_lines`, un diccionario que mapea cada categoría (idioma) a una lista de líneas (nombres). También llevamos un registro de `all_categories` (sólo una lista de idiomas) y `n_categories` para su posterior referencia.

In [ ]:
print(all_categories)

In [ ]:
print(category_lines['Greek'][:5])

## 2. Convirtiendo los nombres a tensores

Ahora que tenemos todos los nombres organizados, necesitamos convertirlos en
Tensores para hacer cualquier uso de ellos.

Para representar una sola **letra**, usamos un "***one-hot vector***" de tamaño `<1 x n_letters>`. Un vector de una letra (***one-hot vector***), consiste en un arreglo que contiene ceros y un (uno solo) en el índice de la letra que se quiere representar. Por ejemplo, `"b"= <0 1 0 0 0 ...>`.

Para representar una **palabra** unimos varios vectores de una letra en una matriz 2D `<line_length x 1 x n_letters>`.

La dimesión 1 extra indica a Pytorch que esa estructura está en lotes (*batch*), indicando que el tamaño del lote es uno.


In [ ]:
import torch

# Encontrar el indice de una letra en all_letters
def letterToIndex(letter):
  return all_letters.find(letter)

# Construir un vector one-hot letter (Ilustrativo)
def letterToTensor(letter):
  tensor = torch.zeros(1, n_letters)
  tensor[0][letterToIndex(letter)] = 1
  return tensor

def lineToTensor(line):
  tensor = torch.zeros(len(line), 1, n_letters)
  for li, letter in enumerate(line):
    tensor[li][0][letterToIndex(letter)] = 1

  return tensor

print(lineToTensor('bekios'))

## 3. Creando la red

Antes del *autograd*, la creación de una red neural recurrente en Torch implicaba
clonar los parámetros de una capa en varios pasos de tiempo. Las capas
mantenían estados y gradientes ocultos que ahora son manejados enteramente por la librería. Esto significa que puedes implementar un RNN de una manera muy "pura", como capas regulares de alimentación...

Este módulo RNN (en su mayoría copiado de "The PyTorch for Torch users")
tutorial <https://pytorch.org/tutorials/beginner/former_torchies/
nn_tutorial.html#example-2-recurrent-net>`__)
son sólo 2 capas lineales que operan en un estado de entrada y oculto, con
una capa de LogSoftmax después de la salida.

Figura:: https://i.imgur.com/Z2xbySO.png

Figura [(softmax)](https://cdn-images-1.medium.com/freeze/max/1000/1*ghDbeGZZUloPOJK7g3OyBg.png?q=20)::



In [ ]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    # Construir componentes
    self.hidden_size = hidden_size
    # Salida de estado oculto
    self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
    # Salida a estimar
    self.i2o = nn.Linear(input_size + hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input, hidden):
    combined = torch.cat((input, hidden), 1) # vector --> [1 x (input + hidden)]
    hidden = self.i2h(combined)
    output = self.i2o(combined)
    output = self.softmax(output)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, self.hidden_size)

n_hidden = 128

# n_letters --> Cantidad de letras que uso para representar mis palabras (57)
# n_hidden --> Tamaño del vector de estado oculto (128)
# n_categories --> La cantidad de paises (18)
rnn = RNN(n_letters, n_hidden, n_categories)

Para ejecutar un paso de esta red necesitamos pasar una entrada (en nuestro caso, el
Tensor para la letra actual) y un estado oculto anterior (que nosotros
inicializar como ceros al principio). Recuperaremos la salida (probabilidad de
cada idioma) y un siguiente estado oculto (que guardamos para el próximo
paso).

In [ ]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)input = letterToIndex('A')
hidden = torch.zeros(1, n_hidden)

# Ejecutar el forwardself.softmax(output)
output, next_hidden = rnn(input, hidden)

print(output)
print(next_hidden)

####
output, next_hidden = rnn(input, hidden)

print(output)
print(next_hidden)

Para mejorar la eficiencia no queremos crear un nuevo Tensor para
cada paso, así que usaremos ``lineToTensor`` en lugar de
``letterToTensor`` y usa *slices*. Esto podría ser optimizado aún más por
precomputando lotes de Tensores.

In [ ]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

print(output)
print(next_hidden)

In [ ]:
input = lineToTensor('Alberto')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[2], hidden)
print(output)
print(next_hidden)

Como puedes ver, la salida es un tensor ``<1 x n_categories>``, donde
cada *item* es la probabilidad de esa categoría (más alta es más probable).

## 4. Entrenando la red

Preparando los datos
--------------------

Antes de entrar en el entrenamiento debemos hacer algunas funciones de ayuda. La primera es interpretar el resultado de la red, que sabemos que es una probabilidad de cada categoría. Podemos usar Tensor.topk para obtener el índice de mayor valor:

In [ ]:
def categoryFromOutput(output):
  top_n, top_i = output.topk(1)
  category_i = top_i[0].item()
  return all_categories[category_i], category_i

print(categoryFromOutput(output))

También queremos una forma rápida de obtener un ejemplo de entrenamiento (un nombre y su
idioma):

In [ ]:
import random

# Elige un elemento de lista en forma aleatoria
def randomChoice(lista):
  return lista[random.randint(0, len(lista)-1)]

def randomTrainingExample():
  category = randomChoice(all_categories)
  # Selecciono la palabra (nombre) de una categoria
  line = randomChoice(category_lines[category])
  # Convertir tensores
  category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
  line_tensor = lineToTensor(line)
  return category, line, category_tensor, line_tensor

print(all_categories)
for i in range(20):
  category, line, category_tensor, line_tensor = randomTrainingExample()
  print('categoria=', category, '/ palabra=', line)

Entrenamiento de la red
--------------------

Ahora todo lo que se necesita para entrenar a esta red es mostrarle un montón de ejemplos.

Para la función de pérdida `n.NLLLoss` es apropiado, ya que la última
La capa del RNN es `nn.LogSoftmax`.

In [ ]:
criterion = nn.NLLLoss()

Cada ciclo de entrenamiento hará:

1. Crear tensores de entrada y de destino

2. Crear un estado inicial oculto igual a cero

3. Leer cada letra y mantener el estado oculto para la próxima letra.

4. Comparar el resultado final con el objetivo

5. *Back-propagate*

6. Devolver la salida y la pérdida

In [ ]:
learning_rate = 0.005

def train(category_tensor, line_tensor):
  # Inicializar la capa hidden en cero
  hidden = rnn.initHidden()

  rnn.zero_grad()

  for i in range(line_tensor.size()[0]):
    output, hidden = rnn(line_tensor[i], hidden)

  loss = criterion(output, category_tensor)
  loss.backward()

  # Actualizar los parametros del modelo
  for p in rnn.parameters():
    p.data.add_(p.grad.data, alpha=-learning_rate)

  return output, loss.item()

Ahora sólo tenemos que correr esto con un montón de ejemplos. Como la función del entrenamiento devuelve tanto la salida como la pérdida, podemos imprimir sus predicciones y también llevar la cuenta de la pérdida. Ya que hay miles de ejemplos, imprimimos sólo cada uno de ellos y tomamos un promedio de la pérdida.

In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000

# Guardamos la perdida
current_loss = 0
all_losses = []

def timeSince(since):
  now = time.time()
  s = now -since
  m = math.floor(s/60)
  s -= m *60
  return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
  # Obtener una categoria y una palabra de forma aleatoria
  category, line, category_tensor, line_tensor = randomTrainingExample()
  output, loss = train(category_tensor, line_tensor)
  current_loss += loss

  # Imprimir los resultados
  if iter % print_every == 0: 
    guess, guess_i = categoryFromOutput(output)
    correct = 'OK' if guess == category else 'ERROR (%s)' % category
    print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter/n_iters * 100, timeSince(start),loss, line, guess, correct))

  if iter % plot_every == 0:
    all_losses.append(current_loss / plot_every)
    current_loss = 0

## 5. Graficando los resultados

Graficar la pérdida histórica de `all_loses` muestra el aprendizaje de la red.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

# 6. Evaluando los resultados

Para ver cómo funciona la red en diferentes categorías, vamos a
crear una **matriz de confusión**, indicando para cada idioma real (filas)
qué idioma adivina la red (columnas). Para calcular la confusión
matriz un montón de muestras se ejecutan a través de la red con
`Evaluar`, que es lo mismo que `Entrenar`, menos el respaldo.

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

Puedes elegir los puntos más brillantes en el eje principal que muestren qué
idiomas que adivina incorrectamente, por ejemplo, el chino para el coreano, y el español para el italiano. Parece que le va muy bien con el griego, y muy mal con Inglés (tal vez debido a la superposición con otros idiomas).

In [ ]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Satoshi')
predict('Bekios')

La versión completa del código la pueden revisar en <https://github.com/spro/practical-pytorch/tree/master/char-rnn-classification>`